## Back Translation && Paraphrasing

In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-paraphraser'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model.cuda();
model.eval();

def paraphrase(text, beams=5, grams=4):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [2]:
def paraphrase_all(commands: list):
    paraphrased = []
    for command in commands:
      paraphrased.append(paraphrase(command))
    return paraphrased

In [3]:
from transformers import MarianMTModel, MarianTokenizer

# load models
language_code = 'sl'
target_model_name = f'Helsinki-NLP/opus-mt-ru-{language_code}'
target_tokenizer = MarianTokenizer.from_pretrained(target_model_name)
target_model = MarianMTModel.from_pretrained(target_model_name)
back_model_name = f'Helsinki-NLP/opus-mt-{language_code}-ru'
back_tokenizer = MarianTokenizer.from_pretrained(back_model_name)
back_model = MarianMTModel.from_pretrained(back_model_name)


def back_translation(batch_texts: list): 
    # translate
    translated_commands = perform_translation(batch_texts, target_model, target_tokenizer, 'sl')
    back_translated_commands = perform_translation(translated_commands, back_model, back_tokenizer, 'ru')
    return back_translated_commands
    


def perform_translation(batch_texts: list, model, tokenizer, target_language: str):
    translated = model.generate(**tokenizer(batch_texts, return_tensors="pt", padding=True))
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    return translated_texts

/usr/local/lib/python3.9/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [4]:
def merge_2(original_commands: list, augmented_commands: list):
    new_items = set(augmented_commands) - set(original_commands)
    merge_result = original_commands + list(new_items)
    return merge_result

def merge(lists_to_merge: list):
  res = []
  for l in lists_to_merge:
    res = merge_2(res, l)
  return res

In [5]:
def first_augmentation(commands: list):
  """ Composition of back translation and paraphrasing """
  commands_sl = back_translation(commands)
  commands_paraphrased = paraphrase_all(commands)
  paraphrase_slovenian = paraphrase_all(commands_sl)
  to_slovenian_paraphrased = back_translation(commands_paraphrased)
  all = merge([commands, commands_paraphrased, commands_sl, paraphrase_slovenian, to_slovenian_paraphrased])
  all = [item.lower() for item in all]
  return all

In [6]:
commands = {
 'move_ship_right' :  
  ['подвинь корабль вправо',
  'плыви вправо',
  'плыву направо',
  'двигаюсь кораблем вправо',
  'право руля',
  'уплыви вправо',
  'отчаливаю направо',
  'судно направо',
  'мы поплывем правее', 
  'на лодке вправо',
  'врубай мотор идем на восток',
  'кораблем направо',
  'я иду кораблем направо',
  'корабль на клетку вправо',
  'шаг правее кораблем'],

'move_ship_left' :  
  ['подвинь корабль влево',
  'плыви налево',
  'плыву влево',
  'двигаюсь кораблем влево',
  'лево руля',
  'уплыви влевво',
  'отчаливаю налево',
  'судно налево',
  'мы поплывем левее', 
  'на лодке налево',
  'врубай мотор идем на запад',
  'кораблем налево',
  'я иду кораблем налево',
  'корабль на клетку влево',
  'шаг левее кораблем'],

'nothing' : 
  ['ты не прав',
  'она права насчет этого', 
  'это мое право', 
  'по правде говоря корабли не летают',
  'люблю чебуреки', 
  'что про лево то сказать', 
  'левшей меньше чем правшей',
  'знаешь мем про лев и прав',
  'как судак и тудак', 
  'хочу морепродуктов',
  'сейчас бы куда-нибудь уплыть',
  'хочу в круиз на корабле',
  'плавание это олимпийский вид спорта',
  'он отчалил вчера на лодке',
  'я рыбачу с лодки свесив удочку направо']}

In [ ]:
commands['move_ship_right'] = first_augmentation(commands['move_ship_right'])
commands['move_ship_left'] = first_augmentation(commands['move_ship_left'])
commands['nothing'] = first_augmentation(commands['nothing'])

### Немного ручной чистки...

In [8]:
commands['move_ship_right'] = ['мы поплывем правее',
 'плыви вправо',
 'право руля',
 'отчаливаю направо',
 'врубай мотор идем на восток',
 'судно направо',
 'шаг правее кораблем',
 'плыву направо',
 'кораблем направо',
 'двигаюсь кораблем вправо',
 'я иду кораблем направо',
 'корабль на клетку вправо',
 'на лодке вправо',
 'подвинь корабль вправо',
 'уплыви вправо',
 'я двигаюсь вправо кораблем',
 'уплывите вправо.',
 'я отчалил направо.',
 'право руля.',
 'мы пойдем лучше.',
 'корабль вправо на клетку',
 'я иду направо.',
 'на корабле направо.',
 'поведи корабль в правую сторону',
 'на правой лодке.',
 'я иду направо кораблем',
 'судно направо.',
 'отплывать вправо',
 'заводи мотор. мы едем на восток.',
 'по правому борту',
 'поверни корабль направо.',
 'отправляюсь направо.',
 'плыви направо.',
 'я двигаюсь на корабле направо.',
 'плыву направо.',
 'корабль направо.',
 'шаг направо от корабля',
 'мы поплывем направо.',
 'я поплыву на корабле направо.',
 'правый рулевой',
 'на борту правого борта.',
 'с кораблём по правому борту',
 'хороший рулевой.',
 'отплыть вправо.',
 'мы поплывём направо налево.',
 'я направо пойду на корабль.',
 'на правом борту.',
 'с кораблем на правом борту',
 'на корабле я направо двигаюсь.',
 'заводи мотор, мы едем на восток.',
 'пляж направо.',
 'правый борт.',
 'я поплыву направо на корабле.',
 'посадка на корабль направо.',
 'на правом катере.',
 'корабль по правому борту к клетке.',
 'мы едем на восток.']


commands['move_ship_left'] = ['на лодке налево',
 'отчаливаю налево',
 'уплыви влевво',
 'я иду кораблем налево',
 'врубай мотор идем на запад',
 'мы поплывем левее',
 'плыви налево',
 'двигаюсь кораблем влево',
 'кораблем налево',
 'плыву влево',
 'корабль на клетку влево',
 'шаг левее кораблем',
 'лево руля',
 'судно налево',
 'подвинь корабль влево',
 'я иду налево кораблем',
 'я отчалил налево.',
 'я двигаюсь влево кораблем',
 'шаг левее корабля',
 'мы пойдем левее.',
 'левый руль.',
 'судно налево.',
 'на левой лодке.',
 'корабль влево на клетку',
 'уплыви влево.',
 'давай давай на запад',
 'налево кораблем.',
 'поведи корабль в левую сторону',
 'корабль налево.',
 'плыви налево.',
 'корабль слева.',
 'на корабле налево.',
 'плыви влево',
 'я двигаюсь на корабле влево.',
 'отправляюсь влево',
 'руль слева',
 'мы поплывем налево.',
 'заводи мотор. мы едем на запад.',
 'передвиньте корабль влево.',
 'на борту левого борта.',
 'я плыву налево.',
 'я полечу на корабле налево.',
 'шаг влево от корабля',
 'передвините корабль в левую сторону.',
 'заводите мотор. мы поедем на западе.',
 'плывай влево.',
 'на левом борту.',
 'мы поплывём на левую сторону.',
 'я налево плыву.',
 'я двигаюсь влево на корабле.',
 'я пойду на корабль влево.',
 'корабль влево на клетку.',
 'на левом корабле.',
 'езжай на запад.',
 'слева от корабля.',
 'я полечу налево на корабле.',
 'левый рулевой.',
 'пляж слева.']

 # nothing's OK

## Defne EDA

In [9]:
import random
from random import shuffle
random.seed(1)

stop_words = ['я', 'мы', 'наш', 'мой', 'сейчас', 'тогда', 'он', 'его',
             'она', 'ее', 'для', 'на', 'в', 'по', 'это', 'этот', 'с', 'от', 'при', 'к',
              'быть', 'право', 'вправо', 'направо', 'правый', 'правее', 'лево', 'влево',
              'налево', 'левый', 'левее', 'слева']

#### Formatting

In [10]:
import re
import pymorphy2

def format_command(command):
    result = ''
    command = command.lower()
    for char in command:
        if char in ' йцукенгшщзхъфывапролджэжячсмитьбюё':
            result += char
    result = re.sub(' +', ' ', result) # delete extra spaces
    if result != '':
      if result[0] == ' ':
          result = result[1:]
      result = to_normal_form(result)
    return result


morph = pymorphy2.MorphAnalyzer()

def to_normal_form(command: str):
    words = command.split(' ')
    new_command = ''
    for word in words:
        new_command += morph.parse(word)[0].normal_form + ' '
    return new_command[:-1]

#### Swap words

In [11]:
def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)
    return new_words

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
    return new_words

#### Replace with synonyms

In [12]:
def replace_with_synonym(words: list, n: int):
    """
    Replaces n random words with a synonym
    """
    new_words = words.copy()
    words_to_replace = list()
    word_position = dict()
    for i, word in enumerate(words):
        if word not in stop_words:
          if is_in_vocab(word):
              words_to_replace.append(word)
        word_position[word] = i
    random.shuffle(words_to_replace)
    
    replaced = 0
    for replace_word in words_to_replace:
        synonyms = get_synonyms(replace_word)
        if len(synonyms) == 0:
            continue
        # pick random synonym
        new_word = format_command(random.choice(synonyms))
        new_words[word_position[replace_word]] = new_word
        replaced += 1
        if replaced == n:
            break

    return new_words

#### Insert words

In [13]:
def random_insertion(words, n):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words)
    return new_words

def add_word(new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words)-1)]
        counter += 1
        if counter >= 10:
            return
        if random_word in stop_words or not is_in_vocab(random_word):
          continue
        synonyms = get_synonyms(random_word)
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)

#### Create pipeline

In [14]:
def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):

    sentence = format_command(sentence)
    words = sentence.split(' ')
    words = [word for word in words if word != '']
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = int(num_aug/4)+1

    #sr
    if (alpha_sr > 0):
        n_sr = max(1, int(alpha_sr*num_words))
        for _ in range(num_new_per_technique):
            a_words = replace_with_synonym(words, n_sr)
            augmented_sentences.append(' '.join(a_words))

    #ri
    if (alpha_ri > 0):
        n_ri = max(1, int(alpha_ri*num_words))
        for _ in range(num_new_per_technique):
            a_words = random_insertion(words, n_ri)
            augmented_sentences.append(' '.join(a_words))

    #rs
    if (alpha_rs > 0):
        n_rs = max(1, int(alpha_rs*num_words))
        for _ in range(num_new_per_technique):
            a_words = random_swap(words, n_rs)
            augmented_sentences.append(' '.join(a_words))

    augmented_sentences = [format_command(sentence) for sentence in augmented_sentences]
    shuffle(augmented_sentences)

    #trim so that we have the desired number of augmented sentences
    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    #append the original sentence
    augmented_sentences.append(sentence)

    return augmented_sentences

### For synonyms (word2vec)

In [15]:
tags_conversion = {
    "A": "ADJ",
    "ADV": "ADV",
    "ADVPRO": "ADV",
    "ANUM": "ADJ",
    "APRO": "DET",
    "COM": "ADJ",
    "CONJ": "SCONJ",
    "INTJ": "INTJ",
    "NONLEX": "X",
    "NUM": "NUM",
    "PART": "PART",
    "PR": "ADP",
    "S": "NOUN",
    "SPRO": "PRON",
    "UNKN": "X",
    "V": "VERB"
}

In [16]:
import gensim.downloader

vectors = gensim.downloader.load('word2vec-ruscorpora-300')

In [17]:
from pymystem3 import Mystem

m = Mystem()

def tag_word(word: str):
    processed = m.analyze(word)[0]
    lemma = processed["analysis"][0]["lex"].lower().strip()
    pos = processed["analysis"][0]["gr"].split(',')[0]
    pos = pos.split('=')[0].strip()
    tagged = lemma+'_'+tags_conversion[pos]
    return tagged

def is_in_vocab(word: str):
  try:
    word = tag_word(word)
    syns = vectors.most_similar(positive=word)
    return True
  except:
    return False
    

def get_synonyms(word: str):
  synonyms_vec = vectors.most_similar(positive=tag_word(word))
  synonyms = [item[0].split('_')[0] for item in synonyms_vec]
  return synonyms

## Augment with EDA



In [18]:
total_commands_count = 0
for intent in commands.keys():
  total_commands_count += len(commands[intent])

In [ ]:
count = 0
progress_percentage = 0
augmented_commands = {}

for intent in commands.keys():
  all = commands[intent]
  augmented_all = []
  for command in all:
    augmented = eda(command)
    augmented_all = merge([augmented_all, augmented])
    
    count += 1
    if round((count / total_commands_count)*100) > progress_percentage:
      progress_percentage = round((count / total_commands_count)*100)
      if progress_percentage % 10 == 0:
        print(f'{progress_percentage}% done...')
  augmented_commands[intent] = augmented_all

## Export

In [20]:
intent_column = []
command_column = []

for intent in augmented_commands.keys():
  for command in augmented_commands[intent]:
    command_column.append(command)
    intent_column.append(intent)

In [21]:
import pandas as pd

df = pd.DataFrame({'command': command_column, 'intent': intent_column})

In [22]:
df

,command,intent
0,на лодка катер вправо,move_ship_right
1,мы бултых направо налево,move_ship_right
2,на эскадра направо,move_ship_right
3,плыть мы поплыть направо,move_ship_right
4,я поплыть направо на судно,move_ship_right
...,...,...
1352,я хотеть судно попасть на корабль,nothing
1353,я хотеть попасть на катер,nothing
1354,я хотеть корабль на попасть,nothing
1355,я хотеть очутиться на корабль,nothing


In [23]:
df.to_csv('commands-w2v-version.csv')

## Currently Unused

### Back Translation: other languages

In [ ]:
def back_traslation(batch_texts: list, language_code: str):
    
    # load models
    target_model_name = f'Helsinki-NLP/opus-mt-ru-{language_code}'
    target_tokenizer = MarianTokenizer.from_pretrained(target_model_name)
    target_model = MarianMTModel.from_pretrained(target_model_name)
    back_model_name = f'Helsinki-NLP/opus-mt-{language_code}-ru'
    back_tokenizer = MarianTokenizer.from_pretrained(back_model_name)
    back_model = MarianMTModel.from_pretrained(back_model_name)
    
    # translate
    translated_commands = perform_translation(original_commands, target_model, target_tokenizer, language_code)
    back_translated_commands = perform_translation(translated_commands, back_model, back_tokenizer, 'ru')
    return back_translated_commands
    


def perform_translation(batch_texts: list, model, tokenizer, target_language: str):
    translated = model.generate(**tokenizer(batch_texts, return_tensors="pt", padding=True))
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    return translated_texts

In [ ]:
original_commands = ['подвинь корабль вправо',
 'плыви вправо',
 'плыву направо',
 'двигаюсь кораблем вправо',
 'право руля',
 'уплыви вправо',
 'отчаливаю направо',
 'судно направо',
 'мы поплывем правее', 
 'на лодке вправо',
 'врубай мотор идем на восток',
 'кораблем направо',
 'я иду кораблем направо',
 'корабль на клетку вправо',
 'шаг правее кораблем']

In [ ]:
# английский
back_traslation(original_commands, 'en')

In [ ]:
# французский
back_traslation(original_commands, 'fr')

In [ ]:
# украинский
back_traslation(original_commands, 'uk')

In [ ]:
# словенский (класс!)
back_traslation(original_commands, 'sl')

In [ ]:
# латышский
back_traslation(original_commands, 'lv')

In [ ]:
augmented_by_sl = merge([original_commands, back_traslation(original_commands, 'sl')])
augmented_by_sl

### Тезаурус RuWordNet

- [Тезаурус для русского языка](https://github.com/avidale/python-ruwordnet)
- Основные сущности:
    - Sense - одно слово/словосочетание с конкретным значением
    - Synset - множество Sense'ов с одинаковыми значениями и  частью речи
- Поддерживаются разлтичные отношения между синсетами:
    <div>
<img src="attachment:image.png" width="700"/>
</div>

In [ ]:
from ruwordnet import RuWordNet

wn = RuWordNet()

В EDA использовались такие методы вместо генерации синонимов word2vec:

In [ ]:
def replace_with_related(words: list, n: int, relation_type: str):
    """
    Replaces n random words with a synonym or hyponym 
    (based on relation_type param).
    """
    new_words = words.copy()
    words_to_replace = list()
    word_position = dict()
    for i, word in enumerate(words):
        if word not in stop_words:
            words_to_replace.append(word)
        word_position[word] = i
    random.shuffle(words_to_replace)
    
    replaced = 0
    for replace_word in words_to_replace:
        related = get_related(replace_word, relation_type)
        if len(related) == 0:
            continue
        # pick random synonym/hyponym
        new_word = format_command(random.choice(related))
        new_words[word_position[replace_word]] = new_word
        replaced += 1
        if replaced == n:
            break

    return new_words

def get_related(word: str, relation_type:str):
    related = []
    if (relation_type == 'synonym'):
        for sense in wn.get_senses(word):
            related.append(sense.synset.title)
    else: # hyponyms
        for sense in wn.get_senses(word):
            for item in sense.synset.hyponyms:
                related.append(item.title)
    return related